In [91]:
import gensim
from gensim.models import Word2Vec, KeyedVectors

In [92]:
import pandas as pd
messages = pd.read_csv("data\SMSSpamCollection.txt", sep="\t", names=["label", "message"])
messages.shape


(5572, 2)

In [93]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [94]:
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
    

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [95]:
nltk.download("wordnet")
corpus = []
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-z]',' ',messages['message'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in stop_words]
    review = " ".join(review)
    corpus.append(review)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [96]:
[[i,j,k] for i,j,k in zip(list(map(len,corpus)),corpus, messages['message']) if i<1]

[[0, '', 'What you doing?how are you?'],
 [0, '', 'Where @'],
 [0, '', '645'],
 [0, '', 'Can a not?'],
 [0, '', ':) '],
 [0, '', 'What you doing?how are you?'],
 [0, '', ':( but your not here....'],
 [0, '', ':-) :-)']]

In [97]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [98]:
words = []
for sent in corpus:
    sent_tokens = sent_tokenize(sent)
    for sent in sent_tokens:
        words.append(simple_preprocess(sent))

In [99]:
model = Word2Vec(words)

In [100]:
model.corpus_count

5564

In [101]:
model.epochs

5

In [102]:
import numpy as np
def avg_word_vec(doc):
    vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

In [103]:
from tqdm import tqdm
X = []
for i in tqdm(range(len(words))):
    X.append(avg_word_vec(words[i]))

100%|██████████| 5564/5564 [00:00<00:00, 6874.17it/s]


In [104]:
len(X)

5564

In [105]:
X[1]

array([-0.06142579,  0.19154847,  0.01323964, -0.02391583,  0.04950835,
       -0.38912642,  0.083449  ,  0.42873192, -0.17028686, -0.17118855,
       -0.16343002, -0.26960996, -0.01714692,  0.09355755,  0.04442863,
       -0.20386492,  0.00986335, -0.20676786, -0.01217867, -0.39185622,
        0.14066431,  0.10197244,  0.02456719, -0.10485889, -0.09056766,
        0.02993118, -0.15993354, -0.22101015, -0.17247972,  0.0411462 ,
        0.23968276,  0.02648398,  0.06438471, -0.07770449, -0.0905517 ,
        0.29264963, -0.05484981, -0.26916865, -0.14929101, -0.4006036 ,
       -0.03639535, -0.17293891,  0.00557013,  0.00111404,  0.1443728 ,
       -0.0683676 , -0.12791401, -0.05308613,  0.16223946,  0.14682251,
        0.11120988, -0.08902498, -0.0590669 ,  0.05586809, -0.16871664,
        0.06948789,  0.15742612,  0.02392611, -0.21779993,  0.05279847,
        0.0588593 ,  0.08564157, -0.08085734, -0.07024577, -0.2765687 ,
        0.2338278 ,  0.03750698,  0.10113424, -0.29234505,  0.25

In [106]:
y = messages[list(map(lambda x: len(x)>0 ,corpus))]
y=pd.get_dummies(y['label'])
y=y.iloc[:,0].values

In [107]:
y.shape

(5564,)

In [108]:
data = []

for i in range(len(X)):
    data.append(np.array(X[i]).reshape(1, -1))
df = pd.DataFrame(np.vstack(data))

In [109]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.073221,0.225716,0.011302,-0.036610,0.061357,-0.451741,0.098024,0.501261,-0.200135,-0.198877,...,0.259595,0.136407,-0.041091,0.055502,0.433653,0.206027,0.108898,-0.302410,0.142055,-0.048006
1,-0.061426,0.191548,0.013240,-0.023916,0.049508,-0.389126,0.083449,0.428732,-0.170287,-0.171189,...,0.224691,0.108061,-0.043834,0.044730,0.367913,0.179729,0.098883,-0.258580,0.125557,-0.041667
2,-0.078526,0.233112,0.005908,-0.031371,0.072046,-0.476780,0.104932,0.523092,-0.206303,-0.198184,...,0.265409,0.135564,-0.043263,0.053759,0.453886,0.218865,0.110548,-0.322941,0.158004,-0.048539
3,-0.103679,0.310999,0.016175,-0.044542,0.086812,-0.648431,0.135371,0.715078,-0.275577,-0.281619,...,0.363191,0.187799,-0.055256,0.083750,0.609868,0.295237,0.154350,-0.427959,0.205814,-0.070466
4,-0.083722,0.251441,0.007230,-0.039734,0.065116,-0.504158,0.111375,0.567061,-0.219943,-0.220846,...,0.289603,0.152118,-0.050393,0.061535,0.489947,0.235459,0.130688,-0.341966,0.165980,-0.056976


In [110]:
df["output"] = y

In [111]:
df["output"] = np.where(df['output'] == True, 1, 0)

In [112]:
df.isnull().sum()

0         0
1         0
2         0
3         0
4         0
         ..
96        0
97        0
98        0
99        0
output    0
Length: 101, dtype: int64

In [113]:
X = df.drop(columns=["output"])
y = df["output"]
X.columns = X.columns.astype(str)


In [114]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((4451, 100), (1113, 100), (4451,), (1113,))

In [115]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
classifier=RandomForestClassifier(n_estimators=100, random_state=0)

In [116]:
classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [117]:
y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.9739442946990117
[[122  26]
 [  3 962]]
              precision    recall  f1-score   support

           0       0.98      0.82      0.89       148
           1       0.97      1.00      0.99       965

    accuracy                           0.97      1113
   macro avg       0.97      0.91      0.94      1113
weighted avg       0.97      0.97      0.97      1113

